In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark").getOrCreate()
import pandas as pd 
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.functions import col, avg, abs, monotonically_increasing_id, ntile, sum, lit, dense_rank, collect_list
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import to_date
from pyspark.sql.types import DateType, IntegerType, StringType, FloatType
import matplotlib.pyplot as plt
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/25 17:40:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/25 17:40:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Create dataFrame 
events=spark.read.format("csv").option("header","true").load(
    "/Users/emilian/Downloads/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz")

In [4]:
events.show(100)

+------+----------+-------+----------+------+-------------------+-------------------+-----+-----+--------+--------+-------+-----+------------+-------+
|ROW_ID|SUBJECT_ID|HADM_ID|ICUSTAY_ID|ITEMID|          CHARTTIME|          STORETIME| CGID|VALUE|VALUENUM|VALUEUOM|WARNING|ERROR|RESULTSTATUS|STOPPED|
+------+----------+-------+----------+------+-------------------+-------------------+-----+-----+--------+--------+-------+-----+------------+-------+
|   788|        36| 165660|    241249|223834|2134-05-12 12:00:00|2134-05-12 13:56:00|17525|   15|      15|   L/min|      0|    0|        null|   null|
|   789|        36| 165660|    241249|223835|2134-05-12 12:00:00|2134-05-12 13:56:00|17525|  100|     100|    null|      0|    0|        null|   null|
|   790|        36| 165660|    241249|224328|2134-05-12 12:00:00|2134-05-12 12:18:00|20823|  .37|     .37|    null|      0|    0|        null|   null|
|   791|        36| 165660|    241249|224329|2134-05-12 12:00:00|2134-05-12 12:19:00|20823|   

In [5]:
temp_diagnosis=spark.read.format("csv").option("header","true").load(
    "/Users/emilian/Downloads/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz")

In [6]:
temp_diagnosis.show(1000)

+------+---------+--------------------+--------------------+
|ROW_ID|ICD9_CODE|         SHORT_TITLE|          LONG_TITLE|
+------+---------+--------------------+--------------------+
|   174|    01166|TB pneumonia-oth ...|Tuberculous pneum...|
|   175|    01170|TB pneumothorax-u...|Tuberculous pneum...|
|   176|    01171|TB pneumothorax-n...|Tuberculous pneum...|
|   177|    01172|TB pneumothorx-ex...|Tuberculous pneum...|
|   178|    01173|TB pneumothorax-m...|Tuberculous pneum...|
|   179|    01174|TB pneumothorax-c...|Tuberculous pneum...|
|   180|    01175|TB pneumothorax-h...|Tuberculous pneum...|
|   181|    01176|TB pneumothorax-o...|Tuberculous pneum...|
|   182|    01180|Pulmonary TB NEC-...|Other specified p...|
|   183|    01181|Pulmonary TB NEC-...|Other specified p...|
|   184|    01182|Pulmon TB NEC-exa...|Other specified p...|
|   185|    01183|Pulmon TB NEC-mic...|Other specified p...|
|   186|    01184|Pulmon TB NEC-cul...|Other specified p...|
|   187|    01185|Pulmon

### Diagnosis

In [7]:
aux_diagnosis = temp_diagnosis.filter(col("LONG_TITLE").rlike("(?i)diabetes")
        ).select("LONG_TITLE").distinct().union(
                
        temp_diagnosis.filter(col("LONG_TITLE").rlike("(?i)chronic")
        ).filter(col("LONG_TITLE").rlike("(?i)kidney")
        ).select("LONG_TITLE").distinct().union(
                
                
        temp_diagnosis.filter(col("LONG_TITLE").rlike("(?i)heart")
        ).filter(col("LONG_TITLE").rlike("(?i)failure|disease")
        ).select("LONG_TITLE").distinct().union(
                
        ### Healthy Diagnosis
        
        temp_diagnosis.filter(col("LONG_TITLE").rlike("(?i)accident|activity|emotional|aftercare|bipolar|counseling")
        ).select("LONG_TITLE").distinct()
        
                
        )
                
        )  
                
        )

diagnosis = temp_diagnosis.join(aux_diagnosis,
            temp_diagnosis.LONG_TITLE == aux_diagnosis.LONG_TITLE,
            "INNER")
diagnosis.show()



+------+---------+--------------------+--------------------+--------------------+
|ROW_ID|ICD9_CODE|         SHORT_TITLE|          LONG_TITLE|          LONG_TITLE|
+------+---------+--------------------+--------------------+--------------------+
|  2267|    25070|DMII circ nt st u...|Diabetes with per...|Diabetes with per...|
|  2260|    25051|DMI ophth nt st u...|Diabetes with oph...|Diabetes with oph...|
|  2299|     2535|  Diabetes insipidus|  Diabetes insipidus|  Diabetes insipidus|
|  5919|     5881|Nephrogen diabete...|Nephrogenic diabe...|Nephrogenic diabe...|
|  2261|    25052| DMII ophth uncntrld|Diabetes with oph...|Diabetes with oph...|
|  1826|    24961|Sec DM neuro unco...|Secondary diabete...|Secondary diabete...|
|  2262|    25053|  DMI ophth uncntrld|Diabetes with oph...|Diabetes with oph...|
|  1597|    25012|DMII ketoacd unco...|Diabetes with ket...|Diabetes with ket...|
| 14474|     V771|Screen-diabetes m...|Screening for dia...|Screening for dia...|
|  1819|    2493

In [8]:
#Create dataFrame 
patients=spark.read.format("csv").option("header","true").load(
    "/Users/emilian/Downloads/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz")

In [9]:
patients.show(1000)

+------+----------+------+-------------------+-------------------+-------------------+-------------------+-----------+
|ROW_ID|SUBJECT_ID|GENDER|                DOB|                DOD|           DOD_HOSP|            DOD_SSN|EXPIRE_FLAG|
+------+----------+------+-------------------+-------------------+-------------------+-------------------+-----------+
|   234|       249|     F|2075-03-13 00:00:00|               null|               null|               null|          0|
|   235|       250|     F|2164-12-27 00:00:00|2188-11-22 00:00:00|2188-11-22 00:00:00|               null|          1|
|   236|       251|     M|2090-03-15 00:00:00|               null|               null|               null|          0|
|   237|       252|     M|2078-03-06 00:00:00|               null|               null|               null|          0|
|   238|       253|     F|2089-11-26 00:00:00|               null|               null|               null|          0|
|   239|       255|     M|2109-08-05 00:00:00|  

In [10]:
#Create dataFrame 
procedures=spark.read.format("csv").option("header","true").load(
    "/Users/emilian/Downloads/mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv.gz")

In [11]:
procedures.show()

+------+----------+-------+-------+---------+
|ROW_ID|SUBJECT_ID|HADM_ID|SEQ_NUM|ICD9_CODE|
+------+----------+-------+-------+---------+
|   944|     62641| 154460|      3|     3404|
|   945|      2592| 130856|      1|     9671|
|   946|      2592| 130856|      2|     3893|
|   947|     55357| 119355|      1|     9672|
|   948|     55357| 119355|      2|     0331|
|   949|     55357| 119355|      3|     3893|
|   950|      9545| 158060|      1|      034|
|   951|     28600| 189217|      1|     3613|
|   952|     28600| 189217|      2|     3615|
|   953|     28600| 189217|      3|     3961|
|   954|     16994| 115444|      1|     3601|
|   955|     16994| 115444|      2|     3606|
|   956|     16994| 115444|      3|     3723|
|   957|     16994| 115444|      4|     8856|
|   958|     16994| 115444|      5|     9671|
|   959|     16994| 115444|      6|     9915|
|   960|     13579| 186290|      1|     3612|
|   961|     13579| 186290|      2|     3615|
|   962|     13579| 186290|      3

In [12]:
#Create dataFrame 
all_items=spark.read.format("csv").option("header","true").load(
    "/Users/emilian/Downloads/mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz")

In [13]:
all_items.show()

+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+
|ROW_ID|ITEMID|               LABEL|ABBREVIATION|DBSOURCE|    LINKSTO|CATEGORY|UNITNAME|PARAM_TYPE|CONCEPTID|
+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+
|   457|   497|Patient controlle...|        null| carevue|chartevents|    null|    null|      null|     null|
|   458|   498|   PCA Lockout (Min)|        null| carevue|chartevents|    null|    null|      null|     null|
|   459|   499|      PCA Medication|        null| carevue|chartevents|    null|    null|      null|     null|
|   460|   500|      PCA Total Dose|        null| carevue|chartevents|    null|    null|      null|     null|
|   461|   501|  PCV Exh Vt (Obser)|        null| carevue|chartevents|    null|    null|      null|     null|
|  1157|   927|           Allergy 2|        null| carevue|chartevents|    null|    null|      null|     null|
|  1158|  

In [14]:
### All Labels

metrics = all_items.filter(col("LABEL").rlike("(?i)heart")
        ).filter(col("LABEL").rlike("(?i)rate|rhythm")
        ).select("LABEL").distinct().union(
                
                
            
        all_items.filter(col("LABEL").rlike("(?i)diastolic|systolic|glucose|height|weight|temperature|sleep|pulse|blood")
                    
        ).select("LABEL")
        )
        
metrics.show()

+--------------------+
|               LABEL|
+--------------------+
|          Heart Rate|
|Heart Rate Alarm ...|
|     fetal heartrate|
|        Heart Rhythm|
|Heart rate Alarm ...|
|   Lowest Heart Rate|
|      blood cultures|
|      BLOOD CULTURES|
|  Clot to Blood Bank|
|Popliteal Pulse [...|
|PostTib. Pulses [...|
|     Previous Weight|
|    Previous WeightF|
|    Pulses Paradoxus|
|   RLE [Temperature]|
|   RUE [Temperature]|
|Radial Pulse [Right]|
| Ulnar Pulse [Right]|
| fingerstick glucose|
| popliteal pulse r/l|
+--------------------+
only showing top 20 rows



23/05/25 17:40:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [15]:
all_items.show()

+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+
|ROW_ID|ITEMID|               LABEL|ABBREVIATION|DBSOURCE|    LINKSTO|CATEGORY|UNITNAME|PARAM_TYPE|CONCEPTID|
+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+
|   457|   497|Patient controlle...|        null| carevue|chartevents|    null|    null|      null|     null|
|   458|   498|   PCA Lockout (Min)|        null| carevue|chartevents|    null|    null|      null|     null|
|   459|   499|      PCA Medication|        null| carevue|chartevents|    null|    null|      null|     null|
|   460|   500|      PCA Total Dose|        null| carevue|chartevents|    null|    null|      null|     null|
|   461|   501|  PCV Exh Vt (Obser)|        null| carevue|chartevents|    null|    null|      null|     null|
|  1157|   927|           Allergy 2|        null| carevue|chartevents|    null|    null|      null|     null|
|  1158|  

In [16]:
items = all_items.join(metrics,
            metrics.LABEL == all_items.LABEL,
            "INNER")
items.show()

+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+--------------------+
|ROW_ID|ITEMID|               LABEL|ABBREVIATION|DBSOURCE|    LINKSTO|CATEGORY|UNITNAME|PARAM_TYPE|CONCEPTID|               LABEL|
+------+------+--------------------+------------+--------+-----------+--------+--------+----------+---------+--------------------+
|  1160|   938|      blood cultures|        null| carevue|chartevents|    null|    null|      null|     null|      blood cultures|
|  1163|   942|      BLOOD CULTURES|        null| carevue|chartevents|    null|    null|      null|     null|      BLOOD CULTURES|
|  1175|   987|  Clot to Blood Bank|        null| carevue|chartevents|    null|    null|      null|     null|  Clot to Blood Bank|
|   503|   546|Popliteal Pulse [...|        null| carevue|chartevents|    null|    null|      null|     null|Popliteal Pulse [...|
|   506|   549|PostTib. Pulses [...|        null| carevue|chartevents|    null|    

In [17]:
patients_events = events.join(patients,
            events.SUBJECT_ID == patients.SUBJECT_ID,
            "inner")

In [18]:
diagnosis.withColumn("ICD9_CODE",diagnosis.ICD9_CODE.cast(StringType()))
procedures.withColumn("ICD9_CODE",procedures.ICD9_CODE.cast(StringType()))

DataFrame[ROW_ID: string, SUBJECT_ID: string, HADM_ID: string, SEQ_NUM: string, ICD9_CODE: string]

In [19]:
items_patients_events = patients_events.join(items,
            patients_events.ITEMID == items.ITEMID,
            "inner")

In [20]:
def dropDupeDfCols(df):
    newcols = []
    dupcols = []

    for i in range(len(df.columns)):
        if df.columns[i] not in newcols:
            newcols.append(df.columns[i])
        else:
            dupcols.append(i)

    df = df.toDF(*[str(i) for i in range(len(df.columns))])
    for dupcol in dupcols:
        df = df.drop(str(dupcol))

    return df.toDF(*newcols)

IPE = dropDupeDfCols(items_patients_events)
#IPE.show()

In [21]:
PIPE = IPE.join(procedures,
            IPE.SUBJECT_ID == procedures.SUBJECT_ID,
            "inner")

#PIPE.show(100)

In [22]:
final = PIPE.join(diagnosis,
            diagnosis.ICD9_CODE == PIPE.ICD9_CODE,
            "inner")

#final.show(100)

In [23]:
FINAL = dropDupeDfCols(final)
raw_data = FINAL.select("SUBJECT_ID","GENDER", "LABEL","VALUE",
                        "VALUENUM", "VALUEUOM", "ABBREVIATION",
                        "UNITNAME", "PARAM_TYPE", "LONG_TITLE",
                        "CHARTTIME").distinct()



In [24]:
#raw_data.withColumn("DAY", trunc("CHARTTIME","DAY"))

In [25]:
raw_data = raw_data.withColumn("VALUENUM", raw_data.VALUENUM.cast(FloatType()))

In [26]:
pre_out = raw_data.groupBy("SUBJECT_ID", "GENDER", "LONG_TITLE", "CHARTTIME"
                           ).pivot("LABEL").avg("VALUENUM")

In [35]:
pre_out.drop("`Unintentional weight loss >10 lbs..`")

DataFrame[SUBJECT_ID: string, GENDER: string, LONG_TITLE: string, CHARTTIME: string, ART BP Diastolic: double, ART BP Systolic: double, ART Blood Pressure Alarm - High: double, ART Blood Pressure Alarm - Low: double, ART Blood Pressure Alarm Source: double, Admission Weight (Kg): double, Admission Weight (lbs.): double, Arterial BP [Diastolic]: double, Arterial BP [Systolic]: double, Arterial Blood Pressure Alarm - High: double, Arterial Blood Pressure Alarm - Low: double, Arterial Blood Pressure diastolic: double, Arterial Blood Pressure mean: double, Arterial Blood Pressure systolic: double, BP Cuff [Diastolic]: double, BP Cuff [Systolic]: double, BP Left Arm [Diastolic]: double, BP Left Arm [Systolic]: double, BP Left Leg [Diastolic]: double, BP Left Leg [Systolic]: double, BP PAL [Systolic]: double, BP Right Arm [Diastolic]: double, BP Right Arm [Systolic]: double, BP Right Leg [Diastolic]: double, BP Right Leg [Systolic]: double, Birth Weight    (kg): double, Birthweight (kg): dou

In [28]:

pre_out = pre_out.withColumn('diastolica',
                coalesce(pre_out['ART BP Diastolic'],pre_out['Arterial Blood Pressure diastolic'],
                         pre_out['Arterial BP [Diastolic]'],pre_out['BP Cuff [Diastolic]'],
                         pre_out['BP Left Arm [Diastolic]'],pre_out['BP Left Leg [Diastolic]'],
                         pre_out['BP Right Arm [Diastolic]'],pre_out['BP Right Leg [Diastolic]'],
                         pre_out['Manual Blood Pressure Diastolic Left'],
                         pre_out['Manual Blood Pressure Diastolic Right'],
                         pre_out['NBP [Diastolic]'],pre_out['Non Invasive Blood Pressure diastolic'],
                         pre_out['PAP [Diastolic]'],pre_out['Pulmonary Artery Pressure diastolic']
                         ).cast(FloatType()))



pre_out = pre_out.withColumn('sistolica',
                coalesce(pre_out['ART BP Systolic'],pre_out['Arterial Blood Pressure systolic'],
                         pre_out['Arterial BP [Systolic]'],pre_out['BP Cuff [Systolic]'],
                         pre_out['BP Left Arm [Systolic]'],pre_out['BP Left Leg [Systolic]'],
                         pre_out['BP PAL [Systolic]'],pre_out['BP Right Arm [Systolic]'],
                         pre_out['BP Right Leg [Systolic]'],pre_out['Manual Blood Pressure Systolic Left'],
                         pre_out['Manual Blood Pressure Systolic Right'],pre_out['NBP [Systolic]'],
                         pre_out['Non Invasive Blood Pressure systolic'],pre_out['PAP [Systolic]'],
                         pre_out['Pulmonary Artery Pressure systolic']
                         ).cast(FloatType()))


#pre_out = pre_out.withColumn('weight',
#                coalesce(pre_out["Admission Weight (Kg)"],pre_out["Present Weight (kg)"],
#                         #((0.453592)*pre_out["Present Weight (lb)"]),((0.453592)*pre_out["Admission Weight (lbs.)"]),
#                         #((0.0283495)*pre_out["Present Weight (oz)"]),pre_out["Previous Weight (kg)"],
#                         pre_out['Previous Weight'],pre_out['Previous WeightF']
#                         ).cast(FloatType()))


pre_out = pre_out.withColumn('temp',
                coalesce(pre_out['Temperature C'],pre_out['Temperature Celsius'],
                         pre_out['Temperature C (calc)'],pre_out['Temp/Iso/Warmer [Temperature, degrees C]'],
                         (((pre_out['Temperature F'])-32)*(5/9)),(((pre_out['Temperature F (calc)'])-32)*(5/9)),
                        (((pre_out['Temperature Fahrenheit'])-32)*(5/9)),pre_out['RLE [Temperature]'],
                        pre_out['RUE [Temperature]'],pre_out['Skin [Temperature]'],pre_out['Skin Temperature'],
                        
                         ).cast(FloatType()))

pre_out = pre_out.withColumn('ritmo_cardiaco',
                coalesce(pre_out['Heart Rate'],pre_out['Heart Rhythm']
                         ).cast(FloatType()))

pre_out = pre_out.withColumn('glucosa',
                coalesce(pre_out['Glucose'],pre_out['Glucose (70-105)'],
                         pre_out['Glucose (serum)'],pre_out['Glucose (whole blood)'],
                         pre_out['Glucose finger stick'],pre_out['Glucose Monitor #'],
                         pre_out['Fingerstick Glucose']
                         ).cast(FloatType()))

pre_out = pre_out.withColumn('altura',
                coalesce((pre_out['Height (cm)']/100),(pre_out['Height']/100)
                         ).cast(FloatType()))


In [36]:
pre_out = pre_out.withColumn('weight',
                coalesce(pre_out["`Admission Weight (Kg)`"],pre_out["`Present Weight (kg)`"],
                         ((0.453592)*pre_out["`Present Weight (lb)`"]),((0.453592)*pre_out["`Admission Weight (lbs.)`"]),
                         ((0.0283495)*pre_out["`Present Weight (oz)`"]),pre_out["`Previous Weight (kg)`"],
                         pre_out['`Previous Weight`'],pre_out['`Previous WeightF`']
                         ).cast(FloatType()))

AnalysisException: Syntax error in attribute name: Unintentional weight loss >10 lbs..

In [37]:
dataset = pre_out.withColumn('Condition', when(col("LONG_TITLE").rlike("(?i)diabetes"), 'Diabetes'
                                     ).when((col("LONG_TITLE").rlike("(?i)chronic")) & (col("LONG_TITLE").rlike("(?i)kidney")), 'Chronic Kidney Disease'
                                    ).when((col("LONG_TITLE").rlike("(?i)heart")) & (col("LONG_TITLE").rlike("(?i)failure|disease")), 'Heart Failure'
                                    ).otherwise("None")
                                    )


                   
                   

In [38]:
dataset = dataset.select("SUBJECT_ID","GENDER","Condition","diastolica","sistolica","ritmo_cardiaco","temp","glucosa","altura")

In [33]:
df_train = dataset.toPandas()
df_train.to_csv("~/Proyecto_Cruz_Roja/data/TrainingData/Diseases.csv")

In [40]:
spark.stop()

In [ ]:
colnames=["SUBJECT_ID","GENDER","Condition","diastolica","sistolica","ritmo_cardiaco","temp","glucosa","altura"] 

df_train=pd.read_csv('pre_out.csv', names=colnames, header=None)
df_train

In [ ]:
df_train['diastolica'] = df_train['diastolica'].fillna(df_train.groupby('SUBJECT_ID')['diastolica'].transform('mean'))
df_train['sistolica'] = df_train['sistolica'].fillna(df_train.groupby('SUBJECT_ID')['sistolica'].transform('mean'))
df_train['ritmo_cardiaco'] = df_train['ritmo_cardiaco'].fillna(df_train.groupby('SUBJECT_ID')['ritmo_cardiaco'].transform('mean'))
df_train['temp'] = df_train['temp'].fillna(df_train.groupby('SUBJECT_ID')['temp'].transform('mean'))
df_train['glucosa'] = df_train['glucosa'].fillna(df_train.groupby('SUBJECT_ID')['glucosa'].transform('mean'))
df_train['altura'] = df_train['altura'].fillna(df_train.groupby('SUBJECT_ID')['altura'].transform('mean'))


df_train = df_train.drop('SUBJECT_ID', axis=1)
df_train.to_csv('~/Proyecto_Cruz_Roja/data/TrainingData/Diseases_v2.csv')